# Les packages nécessaires

In [1]:
import json
import pandas as pd
import yaml
import re
import numpy as np
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

from sklearn.ensemble import RandomForestClassifier

from transformers import BertTokenizer, TFBertForSequenceClassification

import xgboost as xgb

C:\Users\Benjamin\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Importation des données

In [2]:
# Importation données 
f = open('entities.json')
data = json.load(f)

# Et tokens
with open("tokens.yml", "r", encoding="utf-8") as f:
    tokens = yaml.safe_load(f)

In [3]:
data['01c5e5bf-ec54-4a65-a27f-c179feb19e0e/02a4a658-e31b-4f6c-81f6-21e665ad2e1c.jpg'] # Exemple de page

'ⓄBreton ⒻCyrille Ⓜmenuisier Ⓐ25 ⒸGarçon Ⓚfrançaise\nⓅFerazzi ⒻAuguste Ⓜvitrier Ⓐ30 ⒸGarçon ⓀPiémontaise\nⓅMachol ⒻPierre Ⓜvitrier Ⓐ24 ⒸGarçon ⓀPiémontaise\nⓅDesbois ⒻAlexandre Ⓜprop re Ⓐ48 ⒸHomme marié Ⓚfrançaise\nⓄVignat ⒻZélie Ⓜprop re Ⓗsa fe Ⓐ30 Ⓚfrançaise\nⓄHouy ⒻCaroline Ⓜdomestique Ⓐ24 ⒸFille Ⓚfrançaise\nⓄViolet ⒻEsther Ⓜfe de chambre Ⓐ24 ⒸFille Ⓚfrançaise\nⓅDesbroper ⒻFrançois Jules Ⓜaumônier du collège Ⓐ46 ⒸGarçon Ⓚfrançaise\nⓄApelmeau ⒻThérèse Ⓜdomestique Ⓐ49 ⒸFemme mariée Ⓚfrançaise\nⓅde Chaumont Ⓐ40 ⒸHomme marié Ⓚfrançaise\nⓄde Chaumont ⒻMathilde Ⓜprofess Ⓗsa fe Ⓐ30 ⒸFemme mariée Ⓚfrançaise\nⓄde Chaumont ⒻGeorges Ⓗle fils Ⓐ11 ⒸGarçon Ⓚfrançaise\nⓄde Chaumont ⒻHenro Ⓗle fils Ⓐ8 ⒸGarçon Ⓚfrançaise\nⓄde Chaumont ⒻGaston Ⓗle fils Ⓐ5 ⒸGarçon Ⓚfrançaise\nⓄVoisin ⒻAnne Ⓜdomestique Ⓐ24 ⒸFille Ⓚfrançaise\nⓅAllemant ⒻPhilippe Ⓜtailleur Ⓐ45 ⒸHomme marié Ⓚfrançaise\nⓄDutertre ⒻAdèle Ⓜouvrière Ⓗsa femme Ⓐ45 ⒸFemme mariée Ⓚfrançaise\nⓅGaucheron ⒻJoachim Florentin Ⓜpharmacien Ⓐ30 ⒸHomme m

# Mise sous forme de listes

In [4]:
data_list = list(data.values())

def fnt_split(ligne) :
    ligne = ligne.replace(' ', '_')
    sous_chaines = ligne.split('\n')
    liste_de_listes = []

    for sous_chaine in sous_chaines:
        elements = sous_chaine.split(' ')
        liste_de_listes.append(elements)
    return liste_de_listes

data_ligne = [fnt_split(i) for i in data_list]
# Format : data_ligne[num_page][num_ligne]

In [5]:
data_ligne_final = [data_ligne[i][j] for i in range(len(data_ligne)) for j in range(len(data_ligne[i]))]
# Format : data_ligne_final[num_ligne]

In [6]:
data_ligne_final[0] # Exemple de ligne

['ⓄBreton_ⒻCyrille_Ⓜmenuisier_Ⓐ25_ⒸGarçon_Ⓚfrançaise']

In [7]:
# On supprime les lignes vides

data_ligne_final = [i for i in data_ligne_final if len(i[0]) > 2]
len(data_ligne_final)

25079

In [8]:
# On récupère la liste des symboles

symboles = list(tokens.values())
symboles = [list(i.values())[0] for i in symboles]
symboles

['Ⓐ', 'Ⓑ', 'Ⓒ', 'Ⓓ', 'Ⓔ', 'Ⓕ', 'Ⓗ', 'Ⓘ', 'Ⓙ', 'Ⓚ', 'Ⓛ', 'Ⓜ', 'Ⓞ', 'Ⓟ']

In [9]:
def mise_en_liste_par_ligne(element_ligne) :
    # Initialiser une liste vide pour stocker les sous-listes
    sous_listes = []

    # Initialiser une sous-liste vide
    sous_liste_actuelle = []

    # Parcourir chaque caractère dans la liste
    for caractere in element_ligne[0]:
        # Si le caractère est un symbole, ajouter la sous-liste actuelle à la liste des sous-listes et réinitialiser la sous-liste actuelle
        if caractere in symboles:
            if sous_liste_actuelle:
                sous_listes.append(sous_liste_actuelle)
                sous_liste_actuelle = []
        # Ajouter le caractère à la sous-liste actuelle
        sous_liste_actuelle.append(caractere)

    # Ajouter la dernière sous-liste à la liste des sous-listes
    if sous_liste_actuelle:
        sous_listes.append(sous_liste_actuelle)

    # Rejoindre les caractères dans chaque sous-liste en une chaîne de caractères
    sous_listes = [''.join(sous_liste) for sous_liste in sous_listes]
    sous_listes = [i[0:-1] for i in sous_listes[:-1]] + [sous_listes[-1]] # On supprime les underscores de fin d'éléments
    return sous_listes

In [10]:
# Un exemple d'application
liste = ['ⓅCayre_ⒻFrançoise_Ⓗchef_de_maison_Ⓐ20_ⒸFille_Ⓚfrançaise_Ⓘdans_la_commune_Ⓛoccupée_dans_une_autre_commune']
mise_en_liste_par_ligne(liste)

['ⓅCayre',
 'ⒻFrançoise',
 'Ⓗchef_de_maison',
 'Ⓐ20',
 'ⒸFille',
 'Ⓚfrançaise',
 'Ⓘdans_la_commune',
 'Ⓛoccupée_dans_une_autre_commune']

# Puis de dataframe

In [11]:
data_ligne_final_separe = [mise_en_liste_par_ligne(ligne) for ligne in data_ligne_final] # On met sous forme de liste de liste, en séparant les mots

In [12]:
liste_longueurs_lignes = [len(i) for i in data_ligne_final_separe]
max(liste_longueurs_lignes) # La plus grosse ligne

9

In [13]:
df = pd.DataFrame(data_ligne_final_separe, columns = ["Col_" + str(i) for i in range(max(liste_longueurs_lignes))]) # On met sous forme de df
df = df.fillna("RRien") # Pour éviter les bugs après, on met 2 R car on va supprimer le premier ensuite
df

,Col_0,Col_1,Col_2,Col_3,Col_4,Col_5,Col_6,Col_7,Col_8
0,ⓄBreton,ⒻCyrille,Ⓜmenuisier,Ⓐ25,ⒸGarçon,Ⓚfrançaise,RRien,RRien,RRien
1,ⓅFerazzi,ⒻAuguste,Ⓜvitrier,Ⓐ30,ⒸGarçon,ⓀPiémontaise,RRien,RRien,RRien
2,ⓅMachol,ⒻPierre,Ⓜvitrier,Ⓐ24,ⒸGarçon,ⓀPiémontaise,RRien,RRien,RRien
3,ⓅDesbois,ⒻAlexandre,Ⓜprop_re,Ⓐ48,ⒸHomme_marié,Ⓚfrançaise,RRien,RRien,RRien
4,ⓄVignat,ⒻZélie,Ⓜprop_re,Ⓗsa_fe,Ⓐ30,Ⓚfrançaise,RRien,RRien,RRien
...,...,...,...,...,...,...,...,...,...
25074,ⓅChameton-Dideron,ⒻMarie,Ⓗchef,Ⓑ1869,Ⓚidem,ⒾPailharès,RRien,RRien,RRien
25075,ⓅOde,ⒻMarie,Ⓜouv_chaus_res,Ⓗchef,ⒺCara,Ⓑ1863,Ⓚidem,ⒾSt_Naz_en_Royans,RRien
25076,ⓅBerni,ⒻNello,Ⓜmanoeuvre,Ⓗchef,ⒺBaretto,Ⓑ1886,Ⓚitalienne,ⒾCastel,RRien
25077,ⓄBerni-Laureti,ⒻAnnunziata,Ⓗépouse,Ⓑ1887,Ⓚidem,Ⓘidem,RRien,RRien,RRien


In [14]:
# ligne_test = ['ⓅCayre','ⒻFrançoise','Ⓗchef_de_maison','Ⓐ20','ⒸFille','Ⓚfrançaise','Ⓘdans_la_commune','Ⓛoccupée_dans_une_autre_commune']
def trouver_index_colonne_lien(element_ligne) :
    liste_index = [i for i, num in enumerate(element_ligne) if  isinstance(num, str) and
                   num.startswith('Ⓗ') and
                   num != "RRien"] # On récupère le numéro de colonne du chef de famille
    
    if len(liste_index) == 1 :
        return liste_index[0]
    elif len(liste_index) > 1 :
        return liste_index[0]
    return -1

def trouver_element_colonne_lien(element_ligne) :
    liste_index = [i for i, num in enumerate(element_ligne) if  isinstance(num, str) and
                   num.startswith('Ⓗ') and
                   num != "RRien"] # On récupère le numéro de colonne du chef de famille
    
    if len(liste_index) == 1 :
        return element_ligne[liste_index[0]]
    elif len(liste_index) > 1 :
        return element_ligne[liste_index[0]]
    return "RRien"

df['Index_lien'] = df.apply(trouver_index_colonne_lien, axis=1)
df['Element_lien'] = df.apply(trouver_element_colonne_lien, axis=1)

In [15]:
df

,Col_0,Col_1,Col_2,Col_3,Col_4,Col_5,Col_6,Col_7,Col_8,Index_lien,Element_lien
0,ⓄBreton,ⒻCyrille,Ⓜmenuisier,Ⓐ25,ⒸGarçon,Ⓚfrançaise,RRien,RRien,RRien,-1,RRien
1,ⓅFerazzi,ⒻAuguste,Ⓜvitrier,Ⓐ30,ⒸGarçon,ⓀPiémontaise,RRien,RRien,RRien,-1,RRien
2,ⓅMachol,ⒻPierre,Ⓜvitrier,Ⓐ24,ⒸGarçon,ⓀPiémontaise,RRien,RRien,RRien,-1,RRien
3,ⓅDesbois,ⒻAlexandre,Ⓜprop_re,Ⓐ48,ⒸHomme_marié,Ⓚfrançaise,RRien,RRien,RRien,-1,RRien
4,ⓄVignat,ⒻZélie,Ⓜprop_re,Ⓗsa_fe,Ⓐ30,Ⓚfrançaise,RRien,RRien,RRien,3,Ⓗsa_fe
...,...,...,...,...,...,...,...,...,...,...,...
25074,ⓅChameton-Dideron,ⒻMarie,Ⓗchef,Ⓑ1869,Ⓚidem,ⒾPailharès,RRien,RRien,RRien,2,Ⓗchef
25075,ⓅOde,ⒻMarie,Ⓜouv_chaus_res,Ⓗchef,ⒺCara,Ⓑ1863,Ⓚidem,ⒾSt_Naz_en_Royans,RRien,3,Ⓗchef
25076,ⓅBerni,ⒻNello,Ⓜmanoeuvre,Ⓗchef,ⒺBaretto,Ⓑ1886,Ⓚitalienne,ⒾCastel,RRien,3,Ⓗchef
25077,ⓄBerni-Laureti,ⒻAnnunziata,Ⓗépouse,Ⓑ1887,Ⓚidem,Ⓘidem,RRien,RRien,RRien,2,Ⓗépouse


# Création de $X$ et de $Y$

In [16]:
# Pour trouver cette liste : set(df.Element_lien)
liste_elements_possibles = ['ⒽCHEF_DE_MENAGE',
 'ⒽCh._de_M.',
 'ⒽChef',
 'ⒽChef_de_menage',
 'ⒽChef_de_mén',
 'ⒽChef_de_ménage',
 'ⒽSupérieure_et_chef_de_ménage_du_fils_de_sa_fille',
 'Ⓗch._d._m',
 'Ⓗch._d._m.',
 'Ⓗch._d_._m',
 'Ⓗch._d_m',
 'Ⓗch._d_m.',
 'Ⓗch._de_M.',
 'Ⓗch._de_m',
 'Ⓗch._de_m.',
 'Ⓗch._de_m_ge',
 'Ⓗch._de_mge',
 'Ⓗch._de_ménage',
 'Ⓗch._m',
 'Ⓗch.de_ménage',
 'Ⓗch.m',
 'Ⓗch.m.',
 'Ⓗch_d_m',
 'Ⓗch_d_m.',
 'Ⓗch_d_mé',
 'Ⓗch_de_m',
 'Ⓗch_de_m.',
 'Ⓗch_de_m_ge',
 'Ⓗch_de_ménage',
 'Ⓗchef',
 'Ⓗchef_d_m',
 'Ⓗchef_de_f',
 'Ⓗchef_de_f_le',
 'Ⓗchef_de_fam',
 'Ⓗchef_de_famille',
 'Ⓗchef_de_flle',
 'Ⓗchef_de_m',
 'Ⓗchef_de_m.',
 'Ⓗchef_de_m_e',
 'Ⓗchef_de_m_ge',
 'Ⓗchef_de_maison',
 'Ⓗchef_de_mange',
 'Ⓗchef_de_mge',
 'Ⓗchef_de_mé',
 'Ⓗchef_de_mé_age',
 'Ⓗchef_de_méange',
 'Ⓗchef_de_mén',
 'Ⓗchef_de_mén.',
 'Ⓗchef_de_ménage',
 'Ⓗchef_de_ménage,_son_fils',
 'Ⓗchef_de_ménage_et_veuve',
 'Ⓗchef_de_m⁇nage',
 'Ⓗchef_du_ménage',
 'Ⓗchef_m_ge']

In [17]:
df['Est_chef'] = np.where(df['Element_lien'].isin(liste_elements_possibles), 1, 0)

In [18]:
df

,Col_0,Col_1,Col_2,Col_3,Col_4,Col_5,Col_6,Col_7,Col_8,Index_lien,Element_lien,Est_chef
0,ⓄBreton,ⒻCyrille,Ⓜmenuisier,Ⓐ25,ⒸGarçon,Ⓚfrançaise,RRien,RRien,RRien,-1,RRien,0
1,ⓅFerazzi,ⒻAuguste,Ⓜvitrier,Ⓐ30,ⒸGarçon,ⓀPiémontaise,RRien,RRien,RRien,-1,RRien,0
2,ⓅMachol,ⒻPierre,Ⓜvitrier,Ⓐ24,ⒸGarçon,ⓀPiémontaise,RRien,RRien,RRien,-1,RRien,0
3,ⓅDesbois,ⒻAlexandre,Ⓜprop_re,Ⓐ48,ⒸHomme_marié,Ⓚfrançaise,RRien,RRien,RRien,-1,RRien,0
4,ⓄVignat,ⒻZélie,Ⓜprop_re,Ⓗsa_fe,Ⓐ30,Ⓚfrançaise,RRien,RRien,RRien,3,Ⓗsa_fe,0
...,...,...,...,...,...,...,...,...,...,...,...,...
25074,ⓅChameton-Dideron,ⒻMarie,Ⓗchef,Ⓑ1869,Ⓚidem,ⒾPailharès,RRien,RRien,RRien,2,Ⓗchef,1
25075,ⓅOde,ⒻMarie,Ⓜouv_chaus_res,Ⓗchef,ⒺCara,Ⓑ1863,Ⓚidem,ⒾSt_Naz_en_Royans,RRien,3,Ⓗchef,1
25076,ⓅBerni,ⒻNello,Ⓜmanoeuvre,Ⓗchef,ⒺBaretto,Ⓑ1886,Ⓚitalienne,ⒾCastel,RRien,3,Ⓗchef,1
25077,ⓄBerni-Laureti,ⒻAnnunziata,Ⓗépouse,Ⓑ1887,Ⓚidem,Ⓘidem,RRien,RRien,RRien,2,Ⓗépouse,0


In [19]:
df.groupby("Est_chef").count()

,Col_0,Col_1,Col_2,Col_3,Col_4,Col_5,Col_6,Col_7,Col_8,Index_lien,Element_lien
Est_chef,,,,,,,,,,,
0,20445,20445,20445,20445,20445,20445,20445,20445,20445,20445,20445
1,4634,4634,4634,4634,4634,4634,4634,4634,4634,4634,4634


In [20]:
for i in range(max(liste_longueurs_lignes)) :
    df['X_' + str(i)] = df['Col_' + str(i)].str[1:]

In [21]:
liste_cols_X = ['X_' + str(i) for i in range(max(liste_longueurs_lignes))]
X = df[liste_cols_X]
Y = df[['Est_chef']]

In [22]:
X

,X_0,X_1,X_2,X_3,X_4,X_5,X_6,X_7,X_8
0,Breton,Cyrille,menuisier,25,Garçon,française,Rien,Rien,Rien
1,Ferazzi,Auguste,vitrier,30,Garçon,Piémontaise,Rien,Rien,Rien
2,Machol,Pierre,vitrier,24,Garçon,Piémontaise,Rien,Rien,Rien
3,Desbois,Alexandre,prop_re,48,Homme_marié,française,Rien,Rien,Rien
4,Vignat,Zélie,prop_re,sa_fe,30,française,Rien,Rien,Rien
...,...,...,...,...,...,...,...,...,...
25074,Chameton-Dideron,Marie,chef,1869,idem,Pailharès,Rien,Rien,Rien
25075,Ode,Marie,ouv_chaus_res,chef,Cara,1863,idem,St_Naz_en_Royans,Rien
25076,Berni,Nello,manoeuvre,chef,Baretto,1886,italienne,Castel,Rien
25077,Berni-Laureti,Annunziata,épouse,1887,idem,idem,Rien,Rien,Rien


In [23]:
Y

,Est_chef
0,0
1,0
2,0
3,0
4,0
...,...
25074,1
25075,1
25076,1
25077,0


# Modèle de régression logistique

In [24]:
# Encodage des variables catégorielles
encoder = LabelEncoder()
X_encoded = X.apply(encoder.fit_transform)

# Division des données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_encoded, Y, test_size=0.2, random_state=42)

# Initialisation du modèle de régression logistique
model = LogisticRegression()

# Entraînement du modèle
model.fit(X_train, y_train)

# Prédiction sur l'ensemble de test
y_pred = model.predict(X_test)

# Calcul de la précision
accuracy = accuracy_score(y_test, y_pred)
print("Précision de la régression logistique :", accuracy)

Précision de la régression logistique : 0.8110047846889952


C:\Users\Benjamin\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [25]:
y_zeros = np.zeros(len(y_test))
accuracy = accuracy_score(y_test, y_zeros)
print("Précision de la régression logistique :", accuracy)

Précision de la régression logistique : 0.812200956937799


# Modèle de forêt aléatoire

In [26]:
# Encodage des variables catégorielles
encoder = LabelEncoder()
X_encoded = X.apply(encoder.fit_transform)

# Division des données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_encoded, Y, test_size=0.2, random_state=42)


# Initialisation du modèle RandomForestClassifier
rf_model = RandomForestClassifier()

# Entraînement du modèle
rf_model.fit(X_train, y_train)

# Prédiction sur l'ensemble de test
y_pred_rf = rf_model.predict(X_test)

# Calcul de l'accuracy
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print("Précision du RandomForestClassifier :", accuracy_rf)

C:\Users\Benjamin\AppData\Local\Temp\ipykernel_22164\3338338639.py:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_model.fit(X_train, y_train)


Précision du RandomForestClassifier : 0.9986044657097288


# Modèle XGBoost et recherche de paramètres optimaux

In [27]:
# Encodage des variables catégorielles
encoder = LabelEncoder()
X_encoded = X.apply(encoder.fit_transform)

# Division des données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_encoded, Y, test_size=0.2, random_state=42)


# Initialisation du modèle XGBoost
xgb_model = xgb.XGBClassifier()

# Entraînement du modèle
xgb_model.fit(X_train, y_train)

# Prédiction sur l'ensemble de test
y_pred_xgb = xgb_model.predict(X_test)

# Calcul de l'accuracy
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
print("Précision du XGBoost Classifier :", accuracy_xgb)

Précision du XGBoost Classifier : 0.9964114832535885


In [28]:
# Encodage des variables catégorielles
encoder = LabelEncoder()
X_encoded = X.apply(encoder.fit_transform)

param_grid = {
    'learning_rate': [0.1, 0.01],
    'max_depth': [3, 5, 7],
    'n_estimators': [100, 200, 300, 400, 500, 600]
}

# Initialisation du modèle XGBoost
xgb_model = xgb.XGBClassifier()

# Recherche des meilleurs paramètres avec GridSearchCV
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=3, scoring='accuracy')
grid_search.fit(X_encoded, Y)

# Affichage des meilleurs paramètres trouvés
print("Meilleurs paramètres :", grid_search.best_params_)

Meilleurs paramètres : {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 500}


In [29]:
# Maintenant qu'on a trouvé les meilleurs paramètres, on train/test split puis on regarde :
# Encodage des variables catégorielles
encoder = LabelEncoder()
X_encoded = X.apply(encoder.fit_transform)

X_train, X_test, y_train, y_test = train_test_split(X_encoded, Y, test_size=0.2, random_state=42)


# Entraînement du modèle avec les meilleurs paramètres
best_xgb_model = xgb.XGBClassifier(**grid_search.best_params_)
best_xgb_model.fit(X_train, y_train)

# Prédiction sur l'ensemble de test
y_pred_xgb = best_xgb_model.predict(X_test)

# Calcul de l'accuracy
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
print("Précision du XGBoost Classifier :", accuracy_xgb)

Précision du XGBoost Classifier : 0.9950159489633174


# Modèle de deep learning

In [30]:
# Récupérer des listes
X_copy = X.copy()
X_copy['combined_text'] = X_copy.apply(lambda row: ' '.join(row.map(str)), axis=1)
X_input = X_copy['combined_text'].tolist()
Y_input = Y['Est_chef'].tolist()

# Tokenisation des données textuelles
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_input)
X_tokenized = tokenizer.texts_to_sequences(X_input)

# Rembourrage des séquences
max_length = 128
X_padded = pad_sequences(X_tokenized, maxlen=max_length, padding='post')


# Diviser les données en ensembles d'entraînement et de validation
X_train, X_val, Y_train, Y_val = train_test_split(X_padded, Y_input, test_size=0.2, random_state=42)

# Convertir les données en tenseurs TensorFlow
X_train = tf.convert_to_tensor(X_train)
X_val = tf.convert_to_tensor(X_val)
Y_train = tf.convert_to_tensor(Y_train)
Y_val = tf.convert_to_tensor(Y_val)

# Création du modèle
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=64, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entraînement du modèle
model.fit(X_train, Y_train, epochs=15, validation_data=(X_val, Y_val))

# Évaluation du modèle
loss, accuracy = model.evaluate(X_val, Y_val)
print(f'Loss: {loss}, Accuracy: {accuracy}')


Epoch 1/15


C:\Users\Benjamin\anaconda3\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


627/627 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.8250 - loss: 0.4449 - val_accuracy: 0.9673 - val_loss: 0.1058
Epoch 2/15
627/627 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9738 - loss: 0.0910 - val_accuracy: 0.9908 - val_loss: 0.0431
Epoch 3/15
627/627 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9894 - loss: 0.0421 - val_accuracy: 0.9896 - val_loss: 0.0371
Epoch 4/15
627/627 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9913 - loss: 0.0330 - val_accuracy: 0.9940 - val_loss: 0.0263
Epoch 5/15
627/627 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9929 - loss: 0.0296 - val_accuracy: 0.9771 - val_loss: 0.0842
Epoch 6/15
627/627 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9918 - loss: 0.0311 - val_accuracy: 0.9910 - val_loss: 0.0317
Epoch 7/15
627/627 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9940 - loss: 0.0221 - val_accuracy: 0.9918 - val_loss: 0.0324
Epoch 8/15
627/627 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9936 - loss: 0.0247 - val_accuracy: 0.9948 - val_

In [ ]:
# Charger le tokenizer BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Concaténer les features pour les données d'entraînement et de test
train_texts = [" ".join(row.astype(str)) for index, row in X_train.iterrows()]
test_texts = [" ".join(row.astype(str)) for index, row in X_test.iterrows()]

# Tokenisation et rembourrage des séquences pour les données d'entraînement et de test
max_length = 128
train_encodings = tokenizer(train_texts, padding='max_length', truncation=True, max_length=max_length, return_tensors="tf")
test_encodings = tokenizer(test_texts, padding='max_length', truncation=True, max_length=max_length, return_tensors="tf")

# Modèle BERT pré-entraîné pour la classification de séquences
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')

# Compiler le modèle
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entraînement du modèle
model.fit(train_encodings, y_train, epochs=3, batch_size=32, validation_split=0.2)

# Évaluation du modèle
loss, accuracy = model.evaluate(test_encodings, y_test)
print("Précision du modèle :", accuracy)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Epoch 1/3
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


